In [5]:
from itertools import combinations_with_replacement as comb
from pythtb import *
import numpy as np

In [44]:
def get_recip_lat_vecs(model):

    lat_vecs = model.get_lat()

    b = 2 * np.pi * np.linalg.inv(lat_vecs).T
    return b

def get_weights(*nks, model, N_sh=1):
    b_vec = get_recip_lat_vecs(model)
    dk = np.array([b_vec[i]/nk for i, nk in enumerate(nks)])
    dim = b_vec.shape[0]
    Cart_idx = list(comb(range(dim), 2))
    print(dk)
    print(Cart_idx)
    n_comb = len(Cart_idx)

    A = np.zeros((n_comb, N_sh))
    q = np.zeros((n_comb))

    for s in range(N_sh):
        for j, (alpha, beta) in enumerate(Cart_idx): 
            if alpha == beta:
                q[j] = 1
            for k in dk:
                A[j, s] += k[alpha] * k[beta]
    print(q)
    U, D, Vt = np.linalg.svd(A, full_matrices=False)
    w = (Vt.T @ np.linalg.inv(np.diag(D)) @ U.T) @ q

    return w/2

In [45]:
nks = 10, 10, 10
# define lattice vectors
lat=[[1.0, 0.0, 0], [0.0, 1.0, 0], [0, 0, 1.0]]
# define coordinates of orbitals

# make two dimensional tight-binding checkerboard model
model = tb_model(3, 3, lat=lat)

# orbs = model.get_orb()
n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)
lat_vecs = model.get_lat() # lattice vectors

 Orbital positions not specified. I will assume a single orbital at the origin.


In [46]:
lat_vecs[0]

array([1., 0., 0.])

In [47]:
b_vec[0]

array([6.28318531, 0.        , 0.        ])

In [48]:
b_vec[1] @ lat_vecs[1] - 2*np.pi

0.0

In [49]:
b_vec = get_recip_lat_vecs(model)
dk = np.array([b_vec[i]/nk for i, nk in enumerate(nks)])
dk

array([[0.62831853, 0.        , 0.        ],
       [0.        , 0.62831853, 0.        ],
       [0.        , 0.        , 0.62831853]])

In [50]:
w = get_weights(*nks, model=model)
w

[[0.62831853 0.         0.        ]
 [0.         0.62831853 0.        ]
 [0.         0.         0.62831853]]
[(0, 0), (0, 1), (0, 2), (1, 1), (1, 2), (2, 2)]
[1. 0. 0. 1. 0. 1.]


array([2.53302959])

In [51]:
# 3D k-mesh
Z = 6 # number of nearest neighbors in reciprocal lattice
G = 2*np.pi/lat_vecs[0][0] # reciprocal lattice vector
b = G / nks[0]  # spacing between allowed kx
w_b = 3/(Z*b**2) # finite difference weights for cubic lattice
w_b

1.2665147955292224